In [1]:
import sys 
!{sys.executable} -m pip install tensorflow

In [2]:
import sys 
!{sys.executable} -m pip install opencv-python


In [3]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_HAND', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),28,28,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

Using TensorFlow backend.


In [4]:
def load_path(root_path = './valid/XR_HAND', size = 512):
	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'F:\Bone Fracture Data\MURA-v1.1\train\XR_HAND',64)
images=load_image(Path,64)

5543


In [5]:
    def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(2,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(Images,labels,epochs=30,verbose=2)
     
        return model

In [6]:
model=trainModel()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/30
 - 6s - loss: 0.5835 - acc: 0.7307
Epoch 2/30
 - 6s - loss: 0.5659 - acc: 0.7323
Epoch 3/30
 - 6s - loss: 0.5512 - acc: 0.7332
Epoch 4/30
 - 5s - loss: 0.5389 - acc: 0.7364
Epoch 5/30
 - 5s - loss: 0.5205 - acc: 0.7417
Epoch 6/30
 - 6s - loss: 0.5050 - acc: 0.7509
Epoch 7/30
 - 6s - loss: 0.4828 - acc: 0.7667
Epoch 8/30
 - 6s - loss: 0.4651 - acc: 0.7795
Epoch 9/30
 - 6s - loss: 0.4279 - acc: 0.7996
Epoch 10/30
 - 6s - loss: 0.4109 - acc: 0.8099
Epoch 11/30
 - 6s - loss: 0.3731 - acc: 0.8308
Epoch 12/30
 - 6s - loss: 0.3522 - acc: 0.8447
Epoch 13/30
 - 5s - loss: 0.3229 - acc: 0.8625
Epoch 14/30
 - 5s - loss: 0.2929 - acc: 0.8719
Epoch 15/30
 - 8s - loss: 0.2698 - acc: 0.8829
Epoch 16/30
 - 10s - loss: 0.2467 - acc: 0.8912
Epoch 17/30
 - 10s - loss: 0.2189 - acc: 0.9132
Epoch 18/30
 - 10s - 

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               401536    
__________

In [8]:
#Validation of test data
ValidPath,Validlabels=load_path(r'F:\Bone Fracture Data\MURA-v1.1\valid\XR_HAND',64)
Validimages=load_image(ValidPath,64)

460


In [9]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.9863745554633763
Test accuracy: 0.6478261


In [ ]:
model.save('hand.h5')
